In [1]:
!wget http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/piControl/r1i1p1f1/Amon/tas/gn/v20190320/tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc


--2022-05-31 22:53:07--  http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/piControl/r1i1p1f1/Amon/tas/gn/v20190320/tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc
Resolving esgf-data.ucar.edu (esgf-data.ucar.edu)... 128.117.181.74
Connecting to esgf-data.ucar.edu (esgf-data.ucar.edu)|128.117.181.74|:80... connected.
HTTP request sent, awaiting response... 200 200
Length: 146087937 (139M) [application/x-netcdf]
Saving to: ‘tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc.2’

tas_Amon_CESM2_piCo 100%[===================>] 139.32M  28.6MB/s    in 5.4s    

2022-05-31 22:53:13 (25.9 MB/s) - ‘tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc.2’ saved [146087937/146087937]



In [2]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
import numpy as np
import cftime
import statsmodels.api as sm

%run arfit_yw.py 

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [3]:
filename = 'tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc'

In [4]:
ds = xr.open_dataset(filename, decode_times=True, use_cftime=True)

/opt/anaconda3/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


In [5]:
### Find annual means and anomalies ###
ds_ann_mean=ds.groupby('time.year').mean()

In [6]:
ds_ann_anom = ds_ann_mean - ds_ann_mean.mean(dim='year')

In [7]:
## ESTIMATE THE AR PARAMETERS ##
ntime,nlat,nlon = ds_ann_anom['tas'].shape
max_order = 10
max_lag = 51
acf = np.empty((nlon,nlat,max_lag))
acf[:] = np.nan
for ny in range(0,nlat,1):
    for nx in range(0,nlon,1):
            dum = ds_ann_anom.tas[:,ny,nx]
            x = dum.values
            order,phi,aic_ar,res,varp,aicc,aic,bic = arfit_yw(x,max_order)
            rho_0 = np.r_[1, -phi]
            arma_process = sm.tsa.ArmaProcess(rho_0) 
            acf_tmp = sm.tsa.ArmaProcess.acf(arma_process,lags=max_lag) #Theortical 
            acf[nx,ny,:]= acf_tmp

In [ ]:
## NOW ESTIMATE T2 ##
T2 = acf[:,:,0] +2*(acf[:,:,1:]**2).sum(axis=2)

In [ ]:
# Make the figure larger
fig = plt.figure(figsize=(11,11))
clevs=np.arange(1,10,1)
# Set the axes using the specified map projection
ax=plt.axes(projection=ccrs.PlateCarree())

# Add cyclic point to data
data=T2.transpose()
data, lons = add_cyclic_point(data, coord=ds['lon'])


# Make a filled contour plot
cs=ax.contourf(lons, ds['lat'], data,clevs,
            transform = ccrs.PlateCarree())

# Add coastlines
ax.coastlines()

# Define the xticks for longitude
ax.set_xticks(np.arange(-180,181,60), crs=ccrs.PlateCarree())
lon_formatter = cticker.LongitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)

# Define the yticks for latitude
ax.set_yticks(np.arange(-90,91,30), crs=ccrs.PlateCarree())
lat_formatter = cticker.LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)

# Add colorbar
cbar = plt.colorbar(cs,label='years')
plt.title('CESM2: TAS Predictability Timescale')